In [1]:
import pandas as pd
import numpy as np
import os
import glob

## 1. Renaming files with it's activity:

### i. Creating a list with all folders name

In [1]:
rootdir = "D:/Ajit/pip/Raw_data"
folder_list = []
for subdir, dirs, files in os.walk(rootdir):
    folder_list.append(subdir)
del folder_list[0]
folder_list

### ii. Changing files name to its activity inside all folders

In [2]:
for loc_folder in folder_list:
    ne=0        # Non-elimination
    ur=0        # Urination
    df=0        # Defecation
    os.chdir(loc_folder)
    for FileList in glob.glob('*.json'):
        loc_json=loc_folder+'/'+FileList
        tag=''
        df_json=pd.read_json(loc_json,lines=True)
        # These are the activities we are interested in:
        activities = ['non_elimination', 'urination', 'defecation']
        for val in df_json['Tags'][0]:
            if val in activities:
                tag=val
                
                if tag=='non_elimination':
                    ne+=1
                    new_name_csv =tag +str(ne)+ ".csv"
                    my_source_csv =loc_json.replace('json','csv')
                    my_dest_csv =loc_folder +('/')+ new_name_csv
                    os.rename(my_source_csv, my_dest_csv)
                    print(my_dest_csv)

                    new_name_json = tag +str(ne)+ ".json"
                    my_source_json = loc_json
                    my_dest_json = loc_folder +('/')+ new_name_json
                    os.rename(my_source_json, my_dest_json)  


                elif tag=='urination':
                    ur+=1
                    new_name_csv =tag +str(ur)+ ".csv"
                    my_source_csv =loc_json.replace('json','csv')
                    my_dest_csv =loc_folder +('/')+ new_name_csv
                    os.rename(my_source_csv, my_dest_csv)
                    print(my_dest_csv)
    
                    new_name_json = tag +str(ur)+ ".json"
                    my_source_json = loc_json
                    my_dest_json = loc_folder +('/')+ new_name_json
                    os.rename(my_source_json, my_dest_json)


                else:
                    df+=1
                    new_name_csv =tag +str(df)+ ".csv"
                    my_source_csv =loc_json.replace('json','csv')
                    my_dest_csv =loc_folder +('/')+ new_name_csv
                    os.rename(my_source_csv, my_dest_csv)
                    print(my_dest_csv)

                    new_name_json = tag +str(df)+ ".json"
                    my_source_json = loc_json
                    my_dest_json = loc_folder +('/')+ new_name_json
                    os.rename(my_source_json, my_dest_json)               
                
            else:
                continue

## 2. Creation of new csv files with outlier Treatment and all features

In [4]:
import os
rootdir = "D:/Ajit/pip/Raw_data/"
folder_list = []
for subdir, dirs, files in os.walk(rootdir):
    folder_list.append(subdir)
del folder_list[0]

### i. Creating empty folder with folder numbers:

In [5]:
for path in folder_list:
    path=path.replace('Raw_data','sample_labeled9')
    os.mkdir(path)

### ii. Outlier treatment with flooring & capping:

In [6]:
def outlier_capping(col_name):
    Q1 = 0
    Q3 = df[col_name].quantile(0.98)
    IQR = Q3
     
    upper_bound = Q3 + 3.5 * IQR   # Whisker width is 3.5
    no_upper_outlier=len(df[df[col_name]>upper_bound])
    print("number of outlier: "+str(no_upper_outlier))
    df[col_name]=np.where(df[col_name]>upper_bound,upper_bound,df[col_name])

### iii. creating new csv files in created folders with outlier treatment and new features:
#### 1.Differential load
#### 2. Sum of absolute value of differential load
#### 3. Acceleration
#### 4. Velocity
#### 5. Energy
#### 6. Power
#### 7. Momentum

In [3]:
for folder in folder_list:
    path=folder

    for filename in os.listdir(path):
        x=path+'/'+filename
        if filename.endswith(".csv"):
#             print(x) # Print this if you want to track which folder is running while running the code
            df=pd.read_csv(x)
            df_json=pd.read_json(x.replace('csv','json'),lines=True)
# Name, Weight, Start time
            df['name'] = df_json['Name_of_cat'][0]
            df['weight'] = df_json['Weight_of_cat'][0]
            df['start_time'] = df_json['Event_start_time'][0]
# Activity
            activities = ['non_elimination', 'urination', 'defecation']
            tag=""
            for val in df_json['Tags'][0]:
                if val in activities:
                    tag=val

            df['activity']=tag

# difference of load from it's previous load (incremental or decremental load) and converted to abslolute value
            df['diff_lc0'] = df['lc0'].diff().abs()
            df['diff_lc1'] = df['lc1'].diff().abs()
            df['diff_lc2'] = df['lc2'].diff().abs()
            df['diff_lc3'] = df['lc3'].diff().abs()
            
# sum of all the loads as in aggregated datasheet all the load sensor's are corelated so we will work with only sum of all load sensor's differential value
# converting force to newton. It was in gm. Devide with 1000 to convert in kg. Then multiply with g=9.81 to convert it in Newton
            df['diff_load_sum'] = (df['diff_lc0']+df['diff_lc1']+df['diff_lc2']+df['diff_lc3'])*(9.81/1000)
            
# Outlier treatment with Q1=0 (because all differential valus are converted to absolute), Q3=0.98 quantile and whisker width = 3.5 and 
            outlier_capping('diff_load_sum')

# w = Weight of cat in kg
            w = (df_json['Weight_of_cat'][0])/1000
    
## Force = Mass * Acceleration
## Acceleration (a) = Force / Weight of cat (w)
            df['acc']= df['diff_load_sum'].apply(lambda x: x/w)
    
## Jerk = (a2-a1)/0.025         (+ve jerk means jerk felt while acceleration, and _ve jerk means jerk felt while deceleration. We focus only on jerk so, convert it to absolute.)
            df['jerk']= ((df['acc'].diff())/0.025).abs()

## Velocity, v2 = (a1*t) + v1
            df['velo'] = 0
            for i in range (1,len(df)):
                df['velo'][i]=(df['acc'][i]*0.025)+(df['velo'][i-1])              

## Energy in Joule (Kinetic Energy), e = 0.5(w*v**2)       
            df['enr']= df['velo'].apply(lambda x: 0.5*w*(x**2))

## Power in watt = Force * Velocity       
            df['power']= df['velo']*df['diff_load_sum']

## Momentum = Mass * Velocity
            df['mom']=w*df['velo']

            filename = x
            filename=filename.replace('Raw_data','sample_labeled9')

            df.to_csv(filename)

## 3. Creation of Final csv file:
#### Following are the columns:
#### 1. Name
#### 2. Activity
#### 3. Weight
#### 4. Duration 
#### 5. Event time
#### 6. mean of "change in load"
#### 7. Varience
#### 8. Maximum value
#### 9. Skewness
#### 10. Kurtosis
#### 11. 90th percentile
#### 12. Acceleration
#### 13. Jerk
#### 14. Velocity
#### 15. Energy
#### 16. Power
#### 17. Momentum

### i. Creating main csv file with all features name

In [8]:
df_main = pd.DataFrame(columns = ['name' ,'activity' ,'weight', 'duration','start_time' ,'change_in_load', 'variance',  'maximum', 'median', 'skewness', 'kurtosis', '0.9_quantile', 'acceleration', 'jerk', 'velocity', 'energy', 'power', 'momentum'])
df_main

,name,activity,weight,duration,start_time,change_in_load,variance,maximum,median,skewness,kurtosis,0.9_quantile,acceleration,jerk,velocity,energy,power,momentum


### ii. Inserting all featurs value in main csv file

In [9]:
def value_insertion(loc_csv):
    df=pd.read_csv(loc_csv)
    print(loc_csv)
    df_main.loc[len(df_main.index)] = [df['name'][0], df['activity'][0], df['weight'][0], df['timestamp'][len(df)-1], df['start_time'][0], df['diff_load_sum'].mean(), df['diff_load_sum'].var(), df['diff_load_sum'].max(), df['diff_load_sum'].median(), df['diff_load_sum'].skew(), df['diff_load_sum'].kurtosis(), df['diff_load_sum'].quantile(0.9), df['acc'].mean(), df['jerk'].mean(), df['velo'].mean(), df['enr'].mean(), df['power'].mean(), df['mom'].mean()]
    return df_main

In [10]:
rootdir = "D:/Ajit/pip/sample_labeled9"
folder_list = []
for subdir, dirs, files in os.walk(rootdir):
    folder_list.append(subdir)
del folder_list[0]

In [4]:
i=0
for folder in folder_list:
    path=folder
    for filename in os.listdir(path):
        x=path+'/'+filename
#         print(i)  # Print this if you want to track which folder is running while running the code
        value_insertion(x)
        i+=1

### iii. Final Data

In [12]:
df_main.head()

,name,activity,weight,duration,start_time,change_in_load,variance,maximum,median,skewness,kurtosis,0.9_quantile,acceleration,jerk,velocity,energy,power,momentum
0,Ryan,defecation,6933,163.975000,2020-08-06 10:08:05-05:00,1.114670,2.425972,14.11659,0.43164,2.088879,5.125106,3.23730,0.160777,2.623752,18.629980,1422.404601,14.699895,129.161654
1,Ryan,defecation,6451,204.774940,2020-09-08 04:32:58-05:00,0.996088,2.262248,19.98297,0.32373,2.502551,9.150935,3.06072,0.154408,2.854197,20.030623,1626.238512,15.753942,129.217548
2,Ryan,defecation,7240,59.975000,2020-07-16 23:03:12-05:00,0.292591,0.218968,4.16925,0.05886,2.892435,11.742110,0.87309,0.040413,0.638543,1.792097,13.463229,0.355113,12.974780
3,Ryan,defecation,7124,156.775000,2020-08-01 10:19:24-05:00,0.952895,2.215583,12.43908,0.23544,2.385345,6.859419,2.95281,0.133758,2.379447,14.293334,898.360000,9.996573,101.825709
4,Ryan,defecation,7150,183.974939,2020-08-12 10:37:19-05:00,1.174555,3.997131,15.75486,0.14715,2.486553,7.492540,3.96324,0.164273,2.929181,21.458935,2029.562636,17.758225,153.431382


## 4. Adding a column in final data sheet if required:

In [ ]:
rootdir = 'C:/Users/ajit.mishra/Downloads/practice/sample/'
folder_list = []
for subdir, dirs, files in os.walk(rootdir):
    folder_list.append(subdir)
del folder_list[0]

In [ ]:
new_name = input('Enter new column name: ')
used_name = input('Enter the name of used column: ')
df_main[new_name] = ""
i=0
for loc_folder in folder_list:
    os.chdir(loc_folder)
    for FileList in glob.glob('*.csv'):
        loc_csv=loc_folder+'/'+FileList
        print(loc_csv)
        df_csv=pd.read_csv(loc_csv)
        val = df_csv[used_name].max()
        print(val)
        df_main.loc[i,new_name]=val
        print(i)
        i+=1